In [ ]:
!pip install biopython
!pip install scipy
!pip install numpy

In [61]:
from Bio import SeqIO
from Bio.Seq import Seq
from collections import defaultdict
import numpy as np
from scipy.spatial.distance import pdist, squareform
import os

In [62]:
# Failu nuskaitymas
file_paths = [
    "/content/data/mamalian1.fasta", "/content/data/mamalian2.fasta",
    "/content/data/mamalian3.fasta", "/content/data/mamalian4.fasta",
    "/content/data/bacterial1.fasta", "/content/data/bacterial2.fasta",
    "/content/data/bacterial3.fasta", "/content/data/bacterial4.fasta"
]

sequences = []
labels = []

for file_path in file_paths:
    with open(file_path, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            sequences.append(record)
            labels.append(os.path.splitext(os.path.basename(file_path))[0])

In [63]:
# Sudaroma po 6 skaitymo remelius kiekvienai sekai (3 į priekį ir 3 reverse komplementarumo)
# Visos remeliu sekos sujungiamos 5 viena concatenated_frames eilute
def get_concatenated_reading_frames(sequence):

    seq = str(sequence.seq)
    rev_seq = str(sequence.reverse_complement().seq)

    frames = [seq, seq[1:], seq[2:]]

    frames += [rev_seq, rev_seq[1:], rev_seq[2:]]

    concatenated_frames = "".join(frames)
    return concatenated_frames

concatenated_sequences = [get_concatenated_reading_frames(seq_record) for seq_record in sequences]

In [64]:
start_codon = "ATG"
stop_codons = ["TAA", "TAG", "TGA"]

# Funkcija suranda ORF'us tarp start ir stop kodonu
# Tikrinama, ar tarp start ir stop kodono nera tarpiniu stop kodonu
# ORF sekos turi buti ilgesnes nei 100 bp, kad butu pridetos prie rezultatu
# ORF pozicijos tikrinamos nuo toliausiai esancio start kodono iki artimiausio stop kodono
# Surastos sekos sujungiamos i viena concatenated_orfs eilute
def find_and_concatenate_orfs(seq):
    orfs = []
    start_positions = []

    for i in range(0, len(seq) - 2, 3):
        codon = seq[i:i+3]

        if codon == start_codon:
            start_positions.append(i)

        elif codon in stop_codons:
            if start_positions:
                for start in reversed(start_positions):
                    orf = seq[start:i+3]

                    intermediate_seq = seq[start:i]
                    if not any(intermediate_seq[j:j+3] in stop_codons for j in range(0, len(intermediate_seq), 3)):
                        if len(orf) >= 100:
                            orfs.append(orf)

                        start_positions = []
                        break

    concatenated_orfs = "".join(orfs)
    return concatenated_orfs

concatenated_orfs_per_sequence = [find_and_concatenate_orfs(seq) for seq in concatenated_sequences]

In [65]:
# Funkcija vercia ORF sekas i baltymo sekas, naudodama Biopython translate metoda
# Vertimas sustoja ties pirmu stop kodonu (to_stop=True)
# Gautos baltymo sekos saugomos protein_seqs sarase
def translate_concatenated_orfs(concatenated_orfs_list):
    protein_seqs = []
    for concatenated_orf in concatenated_orfs_list:
        coding_dna = Seq(concatenated_orf)
        protein_seq = coding_dna.translate(to_stop=True)
        protein_seqs.append(str(protein_seq))
    return protein_seqs

protein_sequences = translate_concatenated_orfs(concatenated_orfs_per_sequence)

In [66]:
import itertools
from collections import defaultdict

# Visi galimi aminorugsciu kodonai
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

# Sudaromi visi galimi kodonai (po viena aminorugsti) ir dikodonai (aminorugsciu poros)
all_codons = list(amino_acids)
all_dicodons = ["".join(pair) for pair in itertools.product(amino_acids, repeat=2)]

def codon_frequency(protein_sequences):
    codon_freq_list = []
    dicodon_freq_list = []

# Inicializuojami kodonu ir dikodonu skaiciavimo zodynai
    for protein in protein_sequences:
        codon_counts = defaultdict(int, {codon: 0 for codon in all_codons})
        dicodon_counts = defaultdict(int, {dicodon: 0 for dicodon in all_dicodons})

# Kiekvieno kodono (aminorugsties) skaiciavimas baltymo sekoje
        for i in range(len(protein)):
            codon = protein[i]
            codon_counts[codon] += 1

# Kiekvieno dikodono (aminorugsciu poros) skaiciavimas baltymo sekoje
        for i in range(len(protein) - 1):
            dicodon = protein[i:i+2]
            dicodon_counts[dicodon] += 1

# Rezultatai pridedami prie kodonu ir dikodonu dazniu sarasu
        codon_freq_list.append(codon_counts)
        dicodon_freq_list.append(dicodon_counts)

    return codon_freq_list, dicodon_freq_list

codon_freq_list, dicodon_freq_list = codon_frequency(protein_sequences)

In [67]:
# Funkcija normalizuoja kodonu ir dikodonu daznius pagal sekos ilgi
def normalize_frequencies(frequencies_list, total_lengths):
    normalized_list = []
    for i, frequencies in enumerate(frequencies_list):
        normalized_frequencies = defaultdict(int)
# Normalizuojamas kiekvieno kodono daznis padalinant is sekos ilgio
        for codon, count in frequencies.items():
            normalized_frequencies[codon] = count / total_lengths[i]
        normalized_list.append(normalized_frequencies)
    return normalized_list

# Skaiciuojamas kiekvienos baltymo sekos ilgis
sequence_lengths = [len(seq) for seq in protein_sequences]

# Normalizuojami kodonu ir dikodonu dazniai pagal sekos ilgi
codon_freq_list = normalize_frequencies(codon_freq_list, sequence_lengths)
dicodon_freq_list = normalize_frequencies(dicodon_freq_list, sequence_lengths)

In [68]:
import itertools

amino_acids = "ACDEFGHIKLMNPQRSTVWY"
all_codons = list(amino_acids)
all_dicodons = ["".join(pair) for pair in itertools.product(amino_acids, repeat=2)]

#Funkcija pavercia dazniu zodyna i vektoriu su kodonais ar dikodonais. Jei daznio nera, priskiriama 0 reiksme.
def freq_dict_to_vector(freq_dict, all_items):
    vector = [freq_dict.get(item, 0) for item in all_items]
    return vector

# Dazniu zodynai verciami i vektorius visoms sekos kombinacijoms
codon_freq_vectors = [freq_dict_to_vector(freq, all_codons) for freq in codon_freq_list]
dicodon_freq_vectors = [freq_dict_to_vector(freq, all_dicodons) for freq in dicodon_freq_list]

# Apskaiciuoja atstumo matrica tarp visu seku, naudodama dazniu vektorius.
def calculate_distance_matrix(vectors):
    dist_matrix = squareform(pdist(vectors, metric="euclidean"))
    return dist_matrix

codon_dist_matrix = calculate_distance_matrix(codon_freq_vectors)
dicodon_dist_matrix = calculate_distance_matrix(dicodon_freq_vectors)

In [72]:
# Funkcija suformatuoja matrica i Phylip formata
def format_phylip_matrix(matrix, labels):
    n = len(matrix)
    result = [f"{n}"]
    for i, row in enumerate(matrix):
        row_str = " ".join([f"{dist:.3f}" for dist in row])
        result.append(f"{labels[i]} {row_str}")
    return "\n".join(result)

# Dikodonu atstumu matrica suformatuojama i Phylip formata
phylip_dicodon_matrix = format_phylip_matrix(dicodon_dist_matrix, labels)
print("Dikodonu matrica:")
print(phylip_dicodon_matrix)

# Kodonu atstumu matrica suformatuojama i Phylip formata
phylip_codon_matrix = format_phylip_matrix(codon_dist_matrix, labels)
print("\nKodonu matrica:")
print(phylip_codon_matrix)

# Matricos irasomos i failus
with open("/content/codon_distance_matrix.phylip", "w") as f:
    f.write(phylip_codon_matrix)

with open("/content/dicodon_distance_matrix.phylip", "w") as f:
    f.write(phylip_dicodon_matrix)

Dikodonu matrica:
8
mamalian1 0.000 0.172 0.165 0.185 0.198 0.175 0.201 0.168
mamalian2 0.172 0.000 0.169 0.168 0.186 0.172 0.202 0.162
mamalian3 0.165 0.169 0.000 0.190 0.193 0.175 0.214 0.163
mamalian4 0.185 0.168 0.190 0.000 0.189 0.179 0.205 0.167
bacterial1 0.198 0.186 0.193 0.189 0.000 0.174 0.181 0.159
bacterial2 0.175 0.172 0.175 0.179 0.174 0.000 0.194 0.137
bacterial3 0.201 0.202 0.214 0.205 0.181 0.194 0.000 0.181
bacterial4 0.168 0.162 0.163 0.167 0.159 0.137 0.181 0.000

Kodonu matrica:
8
mamalian1 0.000 0.247 0.190 0.310 0.264 0.230 0.307 0.214
mamalian2 0.247 0.000 0.197 0.183 0.199 0.182 0.250 0.172
mamalian3 0.190 0.197 0.000 0.263 0.246 0.222 0.303 0.186
mamalian4 0.310 0.183 0.263 0.000 0.248 0.259 0.308 0.231
bacterial1 0.264 0.199 0.246 0.248 0.000 0.131 0.144 0.136
bacterial2 0.230 0.182 0.222 0.259 0.131 0.000 0.205 0.105
bacterial3 0.307 0.250 0.303 0.308 0.144 0.205 0.000 0.215
bacterial4 0.214 0.172 0.186 0.231 0.136 0.105 0.215 0.000


In [73]:
import numpy as np
from collections import defaultdict

amino_acids = "ACDEFGHIKLMNPQRSTVWY"

codon_name_map = {
    "A": "Alanine", "C": "Cysteine", "D": "Aspartic acid", "E": "Glutamic acid",
    "F": "Phenylalanine", "G": "Glycine", "H": "Histidine", "I": "Isoleucine",
    "K": "Lysine", "L": "Leucine", "M": "Methionine", "N": "Asparagine",
    "P": "Proline", "Q": "Glutamine", "R": "Arginine", "S": "Serine",
    "T": "Threonine", "V": "Valine", "W": "Tryptophan", "Y": "Tyrosine"
}

all_codons = list(amino_acids)

all_dicodons = ["".join(pair) for pair in itertools.product(amino_acids, repeat=2)]

# Funkcija grazina pilna dikodono pavadinima
def dicodon_name(dicodon):
    return f"{codon_name_map[dicodon[0]]}-{codon_name_map[dicodon[1]]}"

# Funkcija skaiciuoja kodonu arba dikodonu dazniu variacija
def calculate_variance(frequency_list, all_items):
    variances = defaultdict(float)

    for item in all_items:
        frequencies = [freq_dict.get(item, 0) for freq_dict in frequency_list]
        variances[item] = np.var(frequencies)
    return variances

codon_variances = calculate_variance(codon_freq_list, all_codons)
dicodon_variances = calculate_variance(dicodon_freq_list, all_dicodons)

# Kodonu ir dikodonu variacijos rusiuojamos didejimo tvarka
sorted_codon_variances = sorted(codon_variances.items(), key=lambda x: x[1], reverse=True)
sorted_dicodon_variances = sorted(dicodon_variances.items(), key=lambda x: x[1], reverse=True)

print("Top 3 labiausiai varuijuojantis kodonai:")
for codon, variance in sorted_codon_variances[:3]:
    full_name = codon_name_map[codon]
    print(f"{codon} ({full_name}): Variance = {variance:.6f}")

print("\nTop 3 labiausiai varuijuojantis dikodonai:")
for dicodon, variance in sorted_dicodon_variances[:3]:
    full_names = dicodon_name(dicodon)
    print(f"{dicodon} ({full_names}): Variance = {variance:.6f}")


Top 3 labiausiai varuijuojantis kodonai:
E (Glutamic acid): Variance = 0.003413
R (Arginine): Variance = 0.003267
K (Lysine): Variance = 0.002589

Top 3 labiausiai varuijuojantis dikodonai:
EL (Glutamic acid-Leucine): Variance = 0.000366
IE (Isoleucine-Glutamic acid): Variance = 0.000366
TE (Threonine-Glutamic acid): Variance = 0.000263
